In [ ]:
import pandas as pd
import warnings
import requests, zipfile, io, os
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
warnings.filterwarnings('ignore')

In [ ]:
# download file from url, extract it and store it in the dest_folder
def download_and_extract(url, dest_folder):
  r = requests.get(url)
  z = zipfile.ZipFile(io.BytesIO(r.content))
  z.extractall(dest_folder)

In [ ]:
# define folders and urls
healthy_path = './healthy'
healthy_url = 'http://wwwp.fc.unesp.br/~papa/pub/datasets/Handpd/NewHealthy/Signal.zip'
parkinson_path = './parkinson'
parkinson_url = 'http://wwwp.fc.unesp.br/~papa/pub/datasets/Handpd/NewPatients/Signal.zip'

In [ ]:
download_and_extract(healthy_url, healthy_path)

In [ ]:
download_and_extract(parkinson_url, parkinson_path)

In [ ]:
# actual downloaded signals
healthy_signals = f'{healthy_path}/Signal/'
parkinson_signals = f'{parkinson_path}/Signal/'

In [ ]:
# define Input and labels (0 = healthy, parkinson = 1)
X = []
y = []
seq = 500

In [ ]:
# create dataset with sequence length 500

# healthy signals
for filename in os.listdir(healthy_signals):
  if filename.endswith('txt'):
    df = pd.read_csv(healthy_signals + filename, sep='\s+', header=None, comment='#')
    count = len(df.index) // seq
    for i in range(count):
      X.append((df.iloc[i * seq : (i + 1) * seq]).to_numpy())
      y.append(0)

# parkinson signals
for filename in os.listdir(parkinson_signals):
  if filename.endswith('txt'):
    df = pd.read_csv(parkinson_signals + filename, sep='\s+', header=None, comment='#')
    count = len(df.index) // seq
    for i in range(count):
      X.append((df.iloc[i * seq : (i + 1) * seq]).to_numpy())
      y.append(1) 


In [ ]:
len(X)

29088

In [ ]:
# convert lists to np arrays
X = np.array(X)
y = np.array(y)

In [ ]:
X.shape

(29088, 500, 6)

In [ ]:
y.shape

(29088,)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_valid = scaler.transform(X_valid.reshape(-1, X_valid.shape[-1])).reshape(X_valid.shape)
X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

In [ ]:
model = keras.models.Sequential([
keras.layers.Conv1D(filters=20, kernel_size=4, strides=2, padding="valid", input_shape=[None, 6]),
keras.layers.GRU(128, return_sequences=True),
keras.layers.GRU(128),
keras.layers.Dense(1, activation="sigmoid")
])


In [ ]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [ ]:
checkpoint_filepath = "model.h5"
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
model.fit(X_train, y_train, epochs = 100, batch_size = 32, validation_data=(X_valid, y_valid), callbacks=[model_checkpoint_callback])

Epoch 1/100
737/737 [==============================] - 69s 93ms/step - loss: 0.0794 - accuracy: 0.9704 - val_loss: 0.2032 - val_accuracy: 0.9309
Epoch 2/100
737/737 [==============================] - 69s 94ms/step - loss: 0.0704 - accuracy: 0.9732 - val_loss: 0.1931 - val_accuracy: 0.9316
Epoch 3/100
737/737 [==============================] - 69s 94ms/step - loss: 0.0573 - accuracy: 0.9795 - val_loss: 0.2291 - val_accuracy: 0.9309
Epoch 4/100
737/737 [==============================] - 70s 95ms/step - loss: 0.0674 - accuracy: 0.9731 - val_loss: 0.2156 - val_accuracy: 0.9354
Epoch 5/100
737/737 [==============================] - 70s 95ms/step - loss: 0.0498 - accuracy: 0.9805 - val_loss: 0.2453 - val_accuracy: 0.9290
Epoch 6/100
737/737 [==============================] - 70s 95ms/step - loss: 0.0510 - accuracy: 0.9813 - val_loss: 0.2180 - val_accuracy: 0.9332
Epoch 7/100
737/737 [==============================] - 70s 95ms/step - loss: 0.0518 - accuracy: 0.9806 - val_loss: 0.2153 - val_ac

KeyboardInterrupt: ignored

In [ ]:
model.load_weights(checkpoint_filepath)

In [ ]:
model.evaluate(X_valid, y_valid)


82/82 [==============================] - 3s 38ms/step - loss: 0.2246 - accuracy: 0.9374


[0.22458793222904205, 0.9373567700386047]

In [ ]:
model.evaluate(X_test, y_test)

91/91 [==============================] - 3s 38ms/step - loss: 0.2972 - accuracy: 0.9230


[0.29718512296676636, 0.9229975938796997]

In [ ]:
!ls

healthy  model.h5  parkinson  sample_data


In [ ]:
from google.colab import files
files.download('model.h5') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
y_test.shape

(2909,)

In [ ]:
y_train[:100]

array([1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1])